# Predict Future Sales

Final project for "How to win a Data Science Competition"

* https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data
* https://www.coursera.org/learn/competitive-data-science/home/welcome

In [232]:
# Importació de mòduls

import numpy as np
import pandas as pd

In [233]:
# Importació de dades

sales = pd.read_csv('competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('competitive-data-science-predict-future-sales/items.csv')
test = pd.read_csv('competitive-data-science-predict-future-sales/test.csv')
categories = pd.read_csv('competitive-data-science-predict-future-sales/item_categories.csv')

## Categories

In [234]:
def category_name2(s):
    # Torna la segona part del nom de la categoria, quan és possible
    s = s.split(' - ')
    if len(s)==2:
        return s[1]
    else:
        return s[0]

In [235]:
# Obtenim subcategories

categories['cat_name1'] = categories.iloc[:,0].apply(lambda s: s.split(' - ')[0])
categories['cat_name2'] = categories.iloc[:,0].apply(category_name2)

In [236]:
categories.head()

,item_category_name,item_category_id,cat_name1,cat_name2
0,PC - Гарнитуры/Наушники,0,PC,Гарнитуры/Наушники
1,Аксессуары - PS2,1,Аксессуары,PS2
2,Аксессуары - PS3,2,Аксессуары,PS3
3,Аксессуары - PS4,3,Аксессуары,PS4
4,Аксессуары - PSP,4,Аксессуары,PSP


## Sales

In [237]:
# Outliers

sales = sales.loc[sales.item_price<10000,:]
sales = sales.loc[sales.item_cnt_day<1500,:]

In [238]:
# Preus negatius

sales = sales.loc[sales.item_price>0,:]

In [239]:
# Devolucions

sales = sales.loc[sales.item_cnt_day > 0,:]

In [240]:
# Agregació de ventes per mes

sales_month = sales\
            .groupby(['date_block_num','item_id','shop_id'], as_index = False)\
            .agg({'item_cnt_day':'sum'})\
            .rename(columns= {'item_cnt_day':'item_cnt'})

## Train dataset

In [241]:
# Generem conjunt train: shop, item, date_block_num

train = test.iloc[:,1:].copy()
train['date_block_num'] = 0

for d in range(1,34):
    df = test.iloc[:,1:].copy()
    df['date_block_num'] = d
    train = train.append(df)

In [242]:
# Generem any i mes

train['year'] = 2013 + train['date_block_num']//12
train['month'] = 1 + train['date_block_num']%12

In [243]:
# Afegim categoria

train = pd.merge(train, items[['item_id','item_category_id']], how='left', on='item_id')

In [244]:
# Afegim subcategories

train = pd.merge(train, categories[['item_category_id','cat_name1','cat_name2']], how='left', on='item_category_id')

In [245]:
# Afegim ventes

train = pd.merge(train, sales_month, how='left', on=['item_id','shop_id','date_block_num'])

train.fillna(value=0, inplace=True)

In [246]:
train.head()

,shop_id,item_id,date_block_num,year,month,item_category_id,cat_name1,cat_name2,item_cnt
0,5,5037,0,2013,1,19,Игры,PS3,0.0
1,5,5320,0,2013,1,55,Музыка,CD локального производства,0.0
2,5,5233,0,2013,1,19,Игры,PS3,0.0
3,5,5232,0,2013,1,23,Игры,XBOX 360,0.0
4,5,5268,0,2013,1,20,Игры,PS4,0.0


## Test dataset

In [247]:
test['year'] = 2015
test['month'] = 11
test['date_block_num'] = 34

test = pd.merge(test, items[['item_id','item_category_id']], how='left', on='item_id')
test = pd.merge(test, categories[['item_category_id','cat_name1','cat_name2']], how='left', on='item_category_id')

In [248]:
test.describe()

,ID,shop_id,item_id,year,month,date_block_num,item_category_id
count,214200.000000,214200.000000,214200.000000,214200.0,214200.0,214200.0,214200.000000
mean,107099.500000,31.642857,11019.398627,2015.0,11.0,34.0,46.309608
std,61834.358168,17.561933,6252.644590,0.0,0.0,0.0,16.716581
min,0.000000,2.000000,30.000000,2015.0,11.0,34.0,0.000000
25%,53549.750000,16.000000,5381.500000,2015.0,11.0,34.0,37.000000
50%,107099.500000,34.500000,11203.000000,2015.0,11.0,34.0,43.000000
75%,160649.250000,47.000000,16071.500000,2015.0,11.0,34.0,58.000000
max,214199.000000,59.000000,22167.000000,2015.0,11.0,34.0,83.000000


## Conjunt de validació

In [249]:
# Finestra temporal en mesos
WINDOW = 12 

# Mes per a les prediccions
TEST_MONTH = 34
LAST_MONTH = TEST_MONTH - 1

In [250]:
# Conjunt d'entrenament i validació
train_sales = train.loc[(train['date_block_num']<LAST_MONTH)&(train['date_block_num']>=(LAST_MONTH-WINDOW)),:]
train_eval = train.loc[train['date_block_num']==LAST_MONTH,:]

# Conjunt d'entrenament per a la predicció
train = train.loc[train['date_block_num']>=(TEST_MONTH-WINDOW),:]

## Feature engineering

In [251]:
# Atribut combinat itemshop

train_sales['itemshop_id'] = train_sales['item_id'] * 100 + train_sales['shop_id']
train_eval['itemshop_id'] = train_eval['item_id'] * 100 + train_eval['shop_id']

In [252]:
# Frequency encoding

cat_feats = ['itemshop_id','item_id','shop_id','item_category_id','cat_name1','cat_name2']

for attrib in cat_feats:
    cnt = train_sales.groupby([attrib], as_index=False)\
                        .agg({'item_cnt':'sum'})\
                        .rename(columns={'item_cnt': (attrib + '_freq')})

    train_sales = pd.merge(train_sales, cnt, how='left', on=attrib)
    train_eval = pd.merge(train_eval, cnt, how='left', on=attrib)

In [253]:
# Eliminem atributs categòrics

train_sales.drop(cat_feats, 1, inplace=True)
train_eval.drop(cat_feats, 1, inplace=True)

In [254]:
# Exportem les dades a CSV

train_sales.to_csv('train_sales.csv', index=False)
train_eval.to_csv('train_eval.csv', index=False)

## Preprocessament train, test

In [255]:
# Preprocessament per a la predicció

train['itemshop_id'] = train['item_id'] * 100 + train['shop_id']
test['itemshop_id'] = test['item_id'] * 100 + test['shop_id']

for attrib in cat_feats:
    cnt = train.groupby([attrib], as_index=False)\
                        .agg({'item_cnt':'sum'})\
                        .rename(columns={'item_cnt': (attrib + '_freq')})

    train = pd.merge(train, cnt, how='left', on=attrib)
    test = pd.merge(test, cnt, how='left', on=attrib)

train.drop(cat_feats, 1, inplace=True)
test.drop(cat_feats, 1, inplace=True)

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)